In [243]:
import numpy as np
import pandas as pd
import sys
import re
sys.path.append('..')
import preprocessing.preprocessor
import importlib
importlib.reload(preprocessing.preprocessor)
from preprocessing.preprocessor import Preprocessor

In [277]:
def load_data_txt(abolute_file_path: str, sep: str, encoding: str = "utf-8"):
    result = []
    ad = []
    with open(abolute_file_path, encoding=encoding) as file:
        for line in file:
            if line == sep:
                result.append(ad)
                ad = []
            else:
                ad.append(line)
    return result


In [69]:
positive = []
negative = []


with open('../data/advertisment_negative.txt', encoding='utf-8') as file:
    for line in file:
        if line == "***\n":
            negative.append(ad)
            ad = []
        else:
            ad.append(line)


In [279]:
df_positive = pd.DataFrame({'text': load_data_txt('../data/advertisment_positive.txt', sep="***\n", encoding='utf-8')})
df_negative = pd.DataFrame({'text': load_data_txt('../data/advertisment_negative.txt', sep="***\n", encoding='utf-8')})

In [78]:
pd.set_option('max_colwidth', 1000)

In [101]:
df = pd.read_csv('./../data/exampleOlxData.csv')

In [257]:
df_positive['text'] = df_positive['text'].apply(lambda str_list: " ".join(str_list))
df_negative['text'] = df_negative['text'].apply(lambda str_list: " ".join(str_list))

In [258]:
df_negative['account_created_on'] = '12-01-2023'
df_negative['post_upload_on'] = '12-01-2023'

In [ ]:
##PIPELINE

In [259]:
preprocessor = Preprocessor(df_negative, '../data/stop_words_polish.txt')

In [260]:
preprocessor.preprocess_data('text', 'account_created_on', 'post_upload_on')
df_negative_preprocessed = preprocessor.df

In [261]:
fe_dict = {'sprzedac': ['sprzedać'], 
          'kupic':['kupić'],
          'konto':['konto'],
          'bankowy':['bankowy'],
          'bankowosc': ['bankowość'],
          'zalozenie_konta':['zapłaca','założeć'],
          'komornik':['komornik'],
          'udostepnia':['udostępnia'],
          'finanse': ['finanse'],
          'ryzyko': ['ryzyko'],
        'bezpieczny':['bezpieczny'],
        'przelew': ['przelew'],
        'procent': ['procent'],
        'kwota': ['kwota'],
      'krypto': ['krypto'],
      'kryptowaluta': ['kryptowaluta'],
      'slupa':['słupa'],
      'slup':['słup'],
      'tylko': ['tylko'],
      'szybko': ['szybko'],
      'zysk': ['zysk'],
      'przelew':['przelew'],
      'dane':['dane'],
      'rozny_bank':['różny','bank'],
      'prowizja':['prowizja'],
    'otworzyc':['otworzyć'],
    'zajecie_komornicze':['zajęć','komornicz'],
    'oferta_ograniczona':['oferta','ograniczyć'],
    'zakladane_w_oddziale':['zakładać','oddział'],
   'brak_blokad':['brak','blokad'],
   'brak_limitu':['limit'],
   'dokumenty_bankowe':['dokument','bank'],
   'umowa':['umowa'],
  'karta': 'karta',
  'sim': ['sim'],
  'kaucja': ['kaucja'],
  'opłata': ['opłata'],
   'skan_dowodu':['skan','dowód'],
   'dluga_wspolpraca':['długi','współpraca'],
  'zbudowac_zaufanie':['zbudować','zaufanie'],
 'gwarancja':['gwarancja'],
 'legalny':['legalny'],
 'pożyczka': ['pożyczka'],
 'powazna_propozycja':['poważny','propozycja'],
 'dziś': ['dziś'],
 'teraz': ['teraz']}

In [262]:
import feature_engineering.feature_extractor
import importlib
importlib.reload(feature_engineering.feature_extractor)
from feature_engineering.feature_extractor import FeatureExtractor

In [263]:
fe = FeatureExtractor(df_negative_preprocessed)
fe.feature_engineering('text', terms_dict=fe_dict)

In [265]:
postprocessor = Preprocessor(fe.df, '../data/stop_words_polish.txt')

In [268]:
postprocessor.postprocess_data('text')
df_negative_postprocessed = postprocessor.df

In [270]:
fe = FeatureExtractor(df_negative_postprocessed)
fe.feature_engineering('text', fe_dict, postprocessed=True)

In [ ]:
### END PIPELINE

In [288]:
df_model = fe.df.copy()

In [289]:
df_model['hours_from_account_creation'] =  (fe.df['account_created_on'] - fe.df['post_upload_on']).dt.components['hours']

In [292]:
df_model['fake'] = 1

In [306]:
X = df_model.drop(['text', 'account_created_on', 'post_upload_on'], axis=1)
y = df_model['fake']

In [307]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X, y)

DecisionTreeClassifier()

In [312]:
import pickle
pickle.dump(dt, open('model.sav', 'wb'))

In [317]:
load_data_txt('../data/falszywe_ogloszenia.txt', sep=";\n")

[]

In [331]:
pd.read_csv('../data/falszywe_ogloszenia_mikolaj.txt', sep=';', header=None, names = ['title', 'text'])
# tmp = pd.read_csv('../data/falszywe_ogloszenia.txt', sep=';', header=None, names = ['text'])

,title,text
0,Praca zdalna - Zarabiaj dużo pieniędzy online!,"Szukamy osób zainteresowanych pracą zdalną, która pozwoli Ci zarabiać tysiące dolarów w krótkim czasie. Nasza firma oferuje niezwykłą możliwość zarobku, bez stresu i bez potrzeby doświadczenia. Wystarczy poświęcić kilka godzin dziennie, aby rozpocząć generowanie olbrzymich zysków. Korzyści:- Zarabiaj 10 000 USD tygodniowo! - Brak limitów zarobków! - Nie musisz mieć żadnych umiejętności ani doświadczenia! - Działaj zdalnie, pracuj z dowolnego miejsca na świecie! - Bezpłatne szkolenie i wsparcie! W celu rozpoczęcia pracy, wymagamy jedynie podania swoich danych finansowych. Prosimy o podanie numeru karty kredytowej, daty ważności, kodu CVV oraz danych dostępu do Twojego konta bankowego. Gwarantujemy bezpieczeństwo i poufność Twoich informacji. Nie przegap tej wyjątkowej okazji! Skorzystaj już teraz i zmień swoje życie finansowe na zawsze! Aby się zgłosić, prosimy o wysłanie e-maila na adres: jobscam@example.com Przypominamy, że ta oferta jest ograniczona czasowo, więc nie zwlekaj!"
1,Praca w renomowanej firmie - Wysokie zarobki i korzyści!,"Jesteśmy wiodącą firmą o światowym zasięgu, poszukującą ambitnych i utalentowanych pracowników. Oferujemy pracę w dynamicznym środowisku, atrakcyjne wynagrodzenie i wiele dodatkowych korzyści. Pracuj z nami i odnieś sukces zawodowy! Wymagania: - Wykształcenie wyższe w dziedzinie finansów lub ekonomii - Doświadczenie w branży minimum 5 lat - Umiejętność zarządzania finansami i inwestycjami - Doskonała komunikacja i umiejętności interpersonalne Oferujemy: - Wysokie wynagrodzenie, premie i udziały w zyskach - Luksusowe samochody służbowe i możliwość podróży służbowych - Elastyczny grafik pracy i możliwość pracy zdalnej- Wsparcie i szkolenia rozwojowe z- Ekscytujące projekty i możliwość awansu w firmie Aby aplikować, prosimy o przesłanie swojego CV, listu motywacyjnego i kopii dokumentów potwierdzających wykształcenie i doświadczenie na adres e-mail: joboffer@example.com. Uwaga: W celu finalizacji procesu rekrutacji, wymagamy również wpłaty kaucji w wysokości 5000 złotych. Kwota ta z..."
2,Zajęcie komornicze - Wyprzedaż przedmiotów o wartości!,"Z przykrością informujemy, że w związku z zajęciem komorniczym musimy zorganizować wyprzedaż naszych przedmiotów o wysokiej wartości. Mamy do zaoferowania unikalne okazje, które nie mogą zostać przegapione! Odkryj prawdziwe perełki i oszczędź ogromne sumy pieniędzy! Dostępne przedmioty: - Luksusowe samochody i motocykle - Biżuteria i zegarki markowych marek - Sprzęt elektroniczny i agd - Meble i dekoracje wnętrz - Antyki i kolekcjonerskie przedmioty Wszystkie przedmioty pochodzą z naszej ekskluzywnej kolekcji i są dostępne w wyjątkowo atrakcyjnych cenach. To niepowtarzalna okazja, aby zdobyć przedmioty, które normalnie są poza zasięgiem finansowym! Serdecznie zapraszamy do naszego magazynu, gdzie możesz obejrzeć i zakupić te fantastyczne przedmioty. Zapewniamy fachową obsługę i możliwość negocjacji cen. Pamiętaj, że ilość przedmiotów jest ograniczona, więc nie zwlekaj! Adres magazynu: ul. Przykładowa 123, Miasto W razie pytań, prosimy o kontakt telefoniczny pod numerem: 123-456-78..."
3,Oferta specjalna - Karta kredytowa z niskim oprocentowaniem!,"Masz dosyć wysokich oprocentowań na swojej obecnej karcie kredytowej? Oto szansa na zmianę! Oferujemy Ci specjalną kartę kredytową z niskim oprocentowaniem i atrakcyjnymi korzyściami. Nasza karta kredytowa zapewni Ci: - Stałą stopę oprocentowania na poziomie 0,99% w ciągu pierwszego roku - Brak opłat rocznych za korzystanie z karty - Elastyczny limit kredytowy, dostosowany do Twoich potrzeb - Program lojalnościowy z nagrodami i cashbackiem- Dostęp do ekskluzywnych ofert i promocji Nie trać czasu na wysokie oprocentowanie i ograniczenia innych kart! Złóż wniosek już dziś i ciesz się korzyściami, jakie oferuje nasza specjalna karta kredytowa. Aplikacja jest prosta i szybka. Wypełnij formularz online na naszej stronie internetowej lub zadzwoń pod numer: 12